# Limpieza y Análisis de Datos con Pandas

Este notebook trabaja con el archivo `screentime_analysis_extended.csv`.

Objetivos:
1. Cargar datos y explorarlos.
2. Detectar y tratar valores nulos.
3. Eliminar duplicados.
4. Arreglar tipos de datos.
5. Limpiar texto.
6. Detectar columnas vacías y borrarlas.
7. Guardar el dataset limpio.
8. Hacer un mini análisis final.

## 1. Importación y exploración inicial

In [1]:
import pandas as pd

# Cargar el CSV (ajusta la ruta si lo tienes en otra carpeta)
df = pd.read_csv("spotify_history.csv")

# Vista rápida de las primeras filas
display(df.head())

# Información general de columnas y tipos
df.info()

# Estadísticas básicas (incluyendo texto)
display(df.describe(include='all'))


,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped
0,2J3n32GeLmMjwuAzyhcSNe,2013-07-08 02:44:34,web player,3185,"Say It, Just Say It",The Mowgli's,Waiting For The Dawn,autoplay,clickrow,False,False
1,1oHxIPqJyvAYHy0PVrDU98,2013-07-08 02:45:37,web player,61865,Drinking from the Bottle (feat. Tinie Tempah),Calvin Harris,18 Months,clickrow,clickrow,False,False
2,487OPlneJNni3NWC8SYqhW,2013-07-08 02:50:24,web player,285386,Born To Die,Lana Del Rey,Born To Die - The Paradise Edition,clickrow,unknown,False,False
3,5IyblF777jLZj1vGHG2UD3,2013-07-08 02:52:40,web player,134022,Off To The Races,Lana Del Rey,Born To Die - The Paradise Edition,trackdone,clickrow,False,False
4,0GgAAB0ZMllFhbNc3mAodO,2013-07-08 03:17:52,web player,0,Half Mast,Empire Of The Sun,Walking On A Dream,clickrow,nextbtn,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149860 entries, 0 to 149859
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   spotify_track_uri  149860 non-null  object
 1   ts                 149860 non-null  object
 2   platform           149860 non-null  object
 3   ms_played          149860 non-null  int64 
 4   track_name         149860 non-null  object
 5   artist_name        149860 non-null  object
 6   album_name         149860 non-null  object
 7   reason_start       149717 non-null  object
 8   reason_end         149743 non-null  object
 9   shuffle            149860 non-null  bool  
 10  skipped            149860 non-null  bool  
dtypes: bool(2), int64(1), object(8)
memory usage: 10.6+ MB


,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped
count,149860,149860,149860,1.498600e+05,149860,149860,149860,149717,149743,149860,149860
unique,16527,140422,6,NaN,13839,4113,7948,13,15,2,2
top,1BLOVHYYlH4JUHQGcpt75R,2018-09-30 21:43:18,android,NaN,Ode To The Mets,The Beatles,The Beatles,trackdone,trackdone,True,False
freq,207,125,139821,NaN,207,13621,2063,76655,77194,111583,141991
mean,NaN,NaN,NaN,1.283166e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.178401e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2.795000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.388400e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2.185070e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Detección y manejo de valores nulos (NaN)

In [ ]:
# Conteo de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())

# Rellenar NaN en columnas numéricas clave con valores por defecto
df.fillna({'screen_time(min)': 0, 'notifications': 0}, inplace=True)

# Eliminar filas que no tengan app o tiempo de pantalla (esenciales para análisis)
df.dropna(subset=['app_name', 'screen_time(min)'], inplace=True)

print('\nDespués de limpieza de NaN:')
print(df.isna().sum())

display(df.head())


## 3. Eliminación de duplicados

In [ ]:
# Detectar filas duplicadas completas
duplicados = df[df.duplicated()]
print("Filas duplicadas detectadas:")
display(duplicados)

# Eliminar duplicados
df.drop_duplicates(inplace=True)

print("Tamaño tras eliminar duplicados:", df.shape)


## 4. Conversión de tipos de datos (`astype`, `to_datetime`)

In [ ]:
print("Tipos ANTES:")
print(df.dtypes)

# Convertir columnas numéricas que puedan venir como texto
df['screen_time(min)'] = df['screen_time(min)'].astype(float)

# notifications puede tener mezcla de enteros y strings -> forzamos conversión segura
df['notifications'] = pd.to_numeric(df['notifications'], errors='coerce').fillna(0).astype(int)

# Convertir fechas a datetime, forzando formato cuando hay mezcla
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

print("\nTipos DESPUÉS:")
print(df.dtypes)

display(df.head())


## 5. Eliminación de columnas vacías

In [ ]:
# Eliminamos columnas que estén completamente vacías
df.dropna(axis=1, how='all', inplace=True)

print("Columnas actuales del DataFrame:")
print(df.columns.tolist())


## 6. Limpieza y normalización de texto (`.str` en Pandas)

In [ ]:
# Normalizar nombres de apps:
# - quitar espacios en blanco
# - poner la primera letra en mayúscula de cada palabra
# - eliminar espacios internos (Instagram  -> Instagram, 'YouTube Music' -> 'Youtubemusic')
df['app_name'] = (
    df['app_name']
      .astype(str)
      .str.strip()
      .str.title()
      .str.replace(' ', '', regex=False)
)

display(df[['app_name']].head(10))

# Buscar todas las filas que contienen "Youtube"
youtube_rows = df[df['app_name'].str.contains('Youtube', case=False, na=False)]
print("Filas con Youtube:")
display(youtube_rows)


## 7. Métodos extra de cadena (`str.len`, `str.startswith`, etc.)

In [ ]:
# Longitud del nombre de la app
df['app_name_len'] = df['app_name'].str.len()

# Apps que empiezan por 'Y'
apps_y = df[df['app_name'].str.startswith('Y', na=False)]

print("Apps que empiezan por 'Y':")
display(apps_y[['app_name', 'app_name_len']].drop_duplicates())

display(df[['app_name', 'app_name_len']].head())


## 8. Detección de tipos mezclados en columnas

In [ ]:
print("Tipos actuales:")
print(df.dtypes)

print("\nTipo real de cada celda en las primeras filas:")
display(df.applymap(type).head())

# Convertir automáticamente al tipo más adecuado (opcional)
df = df.convert_dtypes()

print("\nTipos tras convert_dtypes():")
print(df.dtypes)


## 9. Guardar el dataset limpio

In [ ]:
df.to_csv('screentime_analysis_clean.csv', index=False)
print('Archivo screentime_analysis_clean.csv guardado correctamente.')

## 10. Mini análisis final por aplicación

In [ ]:
# ¿Qué apps tienen más tiempo medio de pantalla?
screen_time_media = (
    df.groupby('app_name')['screen_time(min)']
      .mean()
      .sort_values(ascending=False)
)

print("Tiempo medio de pantalla (min) por app:")
display(screen_time_media)

# ¿Qué apps generan más notificaciones?
notif_media = (
    df.groupby('app_name')['notifications']
      .mean()
      .sort_values(ascending=False)
)

print("Notificaciones medias por app:")
display(notif_media)
